<a href="https://colab.research.google.com/github/ThumbRocket/AI_Expert_Lecture_Files/blob/main/4Week/%EC%82%BC%EC%84%B1%EC%8B%A4%EC%8A%B54%EC%9D%BC%EC%B0%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prerequisite

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd /content/drive/MyDrive/

/content/drive/MyDrive


In [3]:
ls

 Classroom/
'Colab Notebooks'/
 data.xlsx
 Document/
 llama3-8b-tokenizer/
'[Samsung AI Expert Day2] L1 - Bayes Classifier (Student).ipynb'
'[Samsung AI Expert Day2] L2 - Word Embedding + Neural LM (Student).ipynb'
'[Student, Samsung] Day1-1 - Simple Spam Classifier'
 Untitled0.ipynb
 삼성_실습_1일차@
 삼성_실습_2일차@
 삼성_실습_2일차.ipynb
 삼성_실습_3일차/


In [4]:
!pip install openai
!pip install langchain
!pip install langchain_openai
!pip install langchain_community
!pip install datasets
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.5/327.5 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 975.5/975.5 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.8/332.8 kB 41.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.4/127.4 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.6/40.6 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━

In [5]:
import openai
import asyncio
import os
import json
from functools import partial
import pandas as pd
from langchain.prompts import PromptTemplate
from langchain_openai import OpenAI
from langchain_community.callbacks import get_openai_callback
from langchain_community.chat_models import ChatOpenAI
import pprint
pp = pprint.PrettyPrinter(indent=4)

# In-context learning

In [6]:
api_model_name = "/root/vllm_api/Meta-Llama-3-8B"

llm = OpenAI(
            model_name=api_model_name,
            openai_api_base="http://147.47.208.96:9999/v1",
            openai_api_key="EMPTY",
            max_retries=3,
            temperature=0.7,
            max_tokens=500,
        )

## Playground (w/ pre-trained model)

In [7]:
response = llm.invoke("Hello World!")
print(response)

 We are a free, open source, and collaborative community dedicated to creating a great API reference experience. We are not affiliated with any particular technology stack. Learn more about our mission and how we are funded here.
Welcome to our new home! We’ve spent the last year or so building out a new version of the API reference docs with new features, a new design, and more content. We're excited to finally share it with you today. This is part of our overall effort to create a community that supports great API documentation in the open source space. If you have any feedback, please feel free to reach out to us on Twitter or GitHub.
This is the home for the official API documentation for the Fastlane ecosystem. Fastlane is a great tool for building your mobile apps with a focus on automation.


In [8]:
input_text = "Hello!"
print(llm.invoke(input_text))

 My name is Adam and I am a web designer & developer with a strong focus on user experience and user interface design. I have worked as a freelance web designer for over 5 years and have a range of professional skills.
I am available for freelance work and have a range of professional skills.
I am a freelance web designer & developer with a strong focus on user experience and user interface design. I have worked as a freelance web designer for over 5 years and have a range of professional skills.
I have also worked on a couple of personal projects and have also worked with a number of clients. I have worked on a number of personal projects and have also worked with a number of clients. I have worked on a number of personal projects and have also worked with a number of clients.
If you are looking for a freelance web designer & developer, you have come to the right place. You can see my portfolio of work, and get in touch with me. I am always happy to answer any questions you may have a

## Lets get into ICL

In [9]:
zero_shot_example = """Translate English to Korean:
train => """
print(llm.invoke(zero_shot_example, stop=['\n']))

1. 교육 2. 훈련 3. 철도 4. 열차


In [10]:
zero_shot_example2 = """train => """
print(llm.invoke(zero_shot_example2, stop=['\n']))

 td


In [11]:
one_shot_example = """apple => 사과
train =>"""
print(llm.invoke(one_shot_example, stop=['\n']))

 트레인


In [12]:
three_shot_example = """Translate English to Korean:
apple => 사과
ruler => 자
train =>"""
print(llm.invoke(three_shot_example, stop=['\n']))

 기차


## Sentiment classification w/ ICL

### Setting

In [29]:
from datasets import load_dataset
import random

dataset_test = load_dataset('imdb',split='test')
df=pd.DataFrame(dataset_test)
test_df = df.sample(n=5)

In [30]:
print(f"data length: {len(test_df)}")
test_df.head()

data length: 10


,text,label
9469,"Oh man, this movie was toe-cringing bad. Bad l...",0
9224,It wasn't until I looked at the trivia section...,0
7292,The Willies starts late one night as brother's...,0
16180,At first I didn't think I would like this movi...,1
21763,"For comedy to work, there are many factors inv...",1


In [31]:
# check the first data
print(test_df.iloc[0]['text'])

Oh man, this movie was toe-cringing bad. Bad look, awful story...the list of good things is shorter than the list of bad things about this movie.<br /><br />Yes sure it's supposed to be a comedy and all but something tells me that this movie was supposed to be the real deal. The movie most certainly does not start of as a comedy but more as an adventurous movie with fun elements put in it. You can especially tells they were serious with this by the acting, that was in a serious and non-comical way, with the exception of a couple of over-the-top sequences obviously. It seemed to be me that at first they tried to make a real and serious adventurous science-fiction movie, perhaps even a franchise but soon began to noticed how bad it all was and simply decided to insert some more and deliberately overdone comical sequence, to make the movie seem more like a comedy and prevent it from turning into a complete disastrous mess.<br /><br />The movie is a weird attempt to mix adventurous swashbu

### Preprocess

In [46]:
LABEL2LANG = {
    0: "negative",
    1: "positive"
}
LANG2LABEL = {
    "negative": 0,
    "positive": 1
}
# List of dictionary
data = []

for index, row in test_df.iterrows():
    data.append({
        "idx" : index,
        "text": row['text'],
        "label": row['label'],
        "label_lang": LABEL2LANG[row['label']]
    })

### Prompt setting

In [41]:
prompt_text1 = """
This movie is awesome! -> positive
This movie is terrible! -> negative
{text} ->"""


prompt_text2 = """
[Example 1]
text: This movie is awesome!
label: positive
[Example 2]
text: This movie is terrible!
label: negative
[Example 3]
text: {text}
label:"""

## TODO ##
prompt_text3 = """
[Example 1]
text: This movie is awesome!
label: positive
[Example 2]
text: This movie is terrible!
label: negative
[Example 3]
text: {text}
label:"""

prompt_template1 = PromptTemplate(
    input_variables=["text"],
    template=prompt_text1
)
prompt_template2 = PromptTemplate(
    input_variables=["text"],
    template=prompt_text2
)
prompt_template3 = PromptTemplate(
    input_variables=["text"],
    template=prompt_text3
)

### Test

In [52]:
from tqdm import tqdm
from copy import deepcopy

def test_prompt(prompt_template, data):
  answer_list = []
  for d in tqdm(data):
      tmp = deepcopy(d)
      input_text = prompt_template.format(text=d['text'])
      response = llm.invoke(input_text, stop=['\n'])
      tmp['response'] = response
      cleaned_response = response.strip().lower()
      tmp['cleaned_response'] = cleaned_response
      try:
          answer = LANG2LABEL[cleaned_response]
      except:
          print(f"\nidx: {d['idx']}, Error: {cleaned_response}")
          answer = -1
      tmp['answer'] = answer
      answer_list.append(tmp)
  return answer_list

def get_accuracy(data):
  correct = 0
  print("calculating accuracy")
  for d in tqdm(data):
      if d['answer'] == d['label']:
          correct += 1
  return correct / len(data)


In [53]:
answer_list = test_prompt(prompt_template1, data)
print(f"\nAccuracy: {get_accuracy(answer_list)}")

 10%|█         | 1/10 [00:03<00:34,  3.84s/it]


idx: 9469, Error: only for the ending<br /><br />ps: i didn't even add in that the movie is also very sexist, which is very hypocritical since the movie is about a planet on which women and men are equal. the women are constantly being portrayed as sex objects and the movie is also full of sexual innuendoes.<br /><br />2011<br /><br />ps2: another thing i forgot to mention is that i also hated the blatant, not always funny puns. i don't know how many times they were saying things like "beam me up scotty" or "i'll be back" or any other reference to a popular movie, tv-show or game. these puns were so bad that they often ruined the fun.


 50%|█████     | 5/10 [00:04<00:02,  1.85it/s]


idx: 16180, Error: mixed


 80%|████████  | 8/10 [00:05<00:00,  3.16it/s]


idx: 14418, Error: neutral


100%|██████████| 10/10 [00:06<00:00,  1.57it/s]


calculating accuracy


100%|██████████| 10/10 [00:00<00:00, 90006.52it/s]


Accuracy: 0.6


In [54]:
answer_list = test_prompt(prompt_template2, data)
print(f"\nAccuracy: {get_accuracy(answer_list)}")

100%|██████████| 10/10 [00:03<00:00,  2.82it/s]


calculating accuracy


100%|██████████| 10/10 [00:00<00:00, 116185.71it/s]


Accuracy: 1.0


In [55]:
answer_list = test_prompt(prompt_template3, data)
print(f"\nAccuracy: {get_accuracy(answer_list)}")

100%|██████████| 10/10 [00:03<00:00,  2.83it/s]


calculating accuracy


100%|██████████| 10/10 [00:00<00:00, 116832.98it/s]


Accuracy: 1.0


In [56]:
pp.pprint(answer_list[0])

{   'answer': 0,
    'cleaned_response': 'negative',
    'idx': 9469,
    'label': 0,
    'label_lang': 'negative',
    'response': ' negative',
    'text': 'Oh man, this movie was toe-cringing bad. Bad look, awful '
            'story...the list of good things is shorter than the list of bad '
            "things about this movie.<br /><br />Yes sure it's supposed to be "
            'a comedy and all but something tells me that this movie was '
            'supposed to be the real deal. The movie most certainly does not '
            'start of as a comedy but more as an adventurous movie with fun '
            'elements put in it. You can especially tells they were serious '
            'with this by the acting, that was in a serious and non-comical '
            'way, with the exception of a couple of over-the-top sequences '
            'obviously. It seemed to be me that at first they tried to make a '
            'real and serious adventurous science-fiction movie, perhaps even '

### Switch Positive & Negative

In [57]:
SWITCHEDLANG2LABEL = {
    "negative": 1,
    "positive": 0
}

def test_opposite_prompt(prompt_template, data):
  answer_list = []
  for d in tqdm(data):
      tmp = deepcopy(d)
      input_text = prompt_template.format(text=d['text'])
      response = llm.invoke(input_text, stop=['\n'])
      tmp['response'] = response
      cleaned_response = response.strip().lower()
      tmp['cleaned_response'] = cleaned_response
      try:
          answer = SWITCHEDLANG2LABEL[cleaned_response]
      except:
          print(f"\nError: {cleaned_response}")
          answer = -1
      tmp['answer'] = answer
      answer_list.append(tmp)
  return answer_list

In [58]:
opposite_prompt_text1 = """
[Example 1]
text: This movie is awesome!
label: negative
[Example 2]
text: This movie is terrible!
label: positive
[Example 3]
text: {text}
label:"""

## TODO ##
opposite_prompt_text2 = """
[Example 1]
text: This movie is awesome!
label: negative
[Example 2]
text: This movie is terrible!
label: positive
[Example 3]
text: {text}
label:"""

opposite_prompt_template = PromptTemplate(
    input_variables=["text"],
    template=opposite_prompt_text1
)

In [59]:
opposite_answer_list = test_opposite_prompt(opposite_prompt_template, data)
print(f"Accuracy: {get_accuracy(opposite_answer_list)}")

100%|██████████| 10/10 [00:03<00:00,  2.55it/s]



Error: neutral
calculating accuracy


100%|██████████| 10/10 [00:00<00:00, 109798.53it/s]

Accuracy: 0.1


### Symbolic in-context learning

In [74]:
input_text = opposite_prompt_text1.format(text=data[0]['text'])
print(input_text)
response = llm.invoke(input_text, stop=['\n'])
print("[llm answer]", response)


[Example 1]
text: This movie is awesome!
label: negative
[Example 2]
text: This movie is terrible!
label: positive
[Example 3]
text: Oh man, this movie was toe-cringing bad. Bad look, awful story...the list of good things is shorter than the list of bad things about this movie.<br /><br />Yes sure it's supposed to be a comedy and all but something tells me that this movie was supposed to be the real deal. The movie most certainly does not start of as a comedy but more as an adventurous movie with fun elements put in it. You can especially tells they were serious with this by the acting, that was in a serious and non-comical way, with the exception of a couple of over-the-top sequences obviously. It seemed to be me that at first they tried to make a real and serious adventurous science-fiction movie, perhaps even a franchise but soon began to noticed how bad it all was and simply decided to insert some more and deliberately overdone comical sequence, to make the movie seem more like a 

In [60]:
SYMBOLICLANG2LABEL = {
    "bar": 0,
    "foo": 1
}

def test_symbolic_prompt(prompt_template, data):
  answer_list = []
  for d in tqdm(data):
      tmp = deepcopy(d)
      input_text = prompt_template.format(text=d['text'])
      response = llm.invoke(input_text, stop=['\n'])
      tmp['response'] = response
      cleaned_response = response.strip().lower()
      tmp['cleaned_response'] = cleaned_response
      try:
          answer = SYMBOLICLANG2LABEL[cleaned_response]
      except:
          print(f"\nError: {cleaned_response}")
          answer = -1
      tmp['answer'] = answer
      answer_list.append(tmp)
  return answer_list

In [70]:
symbolic_prompt_text = """
[Example 1]
text: This movie is awesome!
label: bar
[Example 2]
text: This movie is terrible!
label: pool
[Example 3]
text: {text}
label:"""

symbolic_prompt_template = PromptTemplate(
    input_variables=["text"],
    template=symbolic_prompt_text
)

In [75]:
input_text = symbolic_prompt_text.format(text=data[0]['text'])
print(input_text)
response = llm.invoke(input_text, stop=['\n'])
print("[llm answer]", response)


[Example 1]
text: This movie is awesome!
label: bar
[Example 2]
text: This movie is terrible!
label: pool
[Example 3]
text: Oh man, this movie was toe-cringing bad. Bad look, awful story...the list of good things is shorter than the list of bad things about this movie.<br /><br />Yes sure it's supposed to be a comedy and all but something tells me that this movie was supposed to be the real deal. The movie most certainly does not start of as a comedy but more as an adventurous movie with fun elements put in it. You can especially tells they were serious with this by the acting, that was in a serious and non-comical way, with the exception of a couple of over-the-top sequences obviously. It seemed to be me that at first they tried to make a real and serious adventurous science-fiction movie, perhaps even a franchise but soon began to noticed how bad it all was and simply decided to insert some more and deliberately overdone comical sequence, to make the movie seem more like a comedy an

In [71]:
symbolic_answer_list = test_symbolic_prompt(symbolic_prompt_template, data)
print(f"Accuracy: {get_accuracy(symbolic_answer_list)}")

100%|██████████| 10/10 [00:02<00:00,  3.62it/s]


calculating accuracy


100%|██████████| 10/10 [00:00<00:00, 105916.77it/s]

Accuracy: 0.6


# Chain-of-Thought

In [248]:
api_model_name = "/root/vllm_api/Meta-Llama-3-8B-instruct"#"/root/vllm_api/Meta-Llama-3-8B"

llm = OpenAI(
            model_name=api_model_name,
            openai_api_base="http://147.47.208.83:9999/v1", # 83, 93, 96
            openai_api_key="EMPTY",
            max_retries=3,
            temperature=0.7,
            max_tokens=500,
        )

## Arithmetic task

* Compare direct & CoT

In [157]:
from datasets import load_dataset
import random
gsm8k_main = load_dataset("gsm8k", "main")['test']
print(f"total length of gsm8k: {len(gsm8k_main)}")
sample_indices = random.sample([i for i in range(len(gsm8k_main))], 10)
gsm8k = []
for d in sample_indices:
    gsm8k.append(gsm8k_main[d])
print(f"total length of sampled data of gsm8k: {len(gsm8k)}")

total length of gsm8k: 1319
total length of sampled data of gsm8k: 10


In [158]:
## Example
print(gsm8k[0]['question'])
print("-"*10)
print(gsm8k[0]['answer'])

A team of 4 painters worked on a mansion for 3/8ths of a day every day for 3 weeks. How many hours of work did each painter put in?
----------
There are 24 hours in a day so 3/8ths of a day is (3/8)*24 = <<3/8*24=9>>9 hours
One week has 7 days so 3 weeks have 7*3 = <<7*3=21>>21 days
Each painter put in 9*21 = <<9*21=189>>189 hours of work
#### 189


### Prompt generation

In [159]:
direct_prompt_text = """Solve the following mathematical question and generate ONLY the answer after a tag, 'Answer:' without any rationale.
[Example 1]
Question:Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?
Answer:72

[Example 2]
Question:Weng earns $12 an hour for babysitting. Yesterday, she just did 50 minutes of babysitting. How much did she earn?
Answer:10

[Example 3]
Question:Betty is saving money for a new wallet which costs $100. Betty has only half of the money she needs. Her parents decided to give her $15 for that purpose, and her grandparents twice as much as her parents. How much more money does Betty need to buy the wallet?
Answer:5

[Example 4]
Question:{question}"""

direct_prompt_template = PromptTemplate(
    input_variables=["question"],
    template=direct_prompt_text
)

In [160]:
cot_prompt_text = """Generate the answer for the given mathematical question with step-by-step rationale toward the answer. Provide your final answer based on the rationale using an identifier '####'.
[Example 1]
Question:Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?
Output:Natalia sold 48/2 = <<48/2=24>>24 clips in May.
Natalia sold 48+24 = <<48+24=72>>72 clips altogether in April and May.
#### 72

[Example 2]
Question:Weng earns $12 an hour for babysitting. Yesterday, she just did 50 minutes of babysitting. How much did she earn?
Output:Weng earns 12/60 = $<<12/60=0.2>>0.2 per minute.
Working 50 minutes, she earned 0.2 x 50 = $<<0.2*50=10>>10.
#### 10

[Example 3]
Question:Betty is saving money for a new wallet which costs $100. Betty has only half of the money she needs. Her parents decided to give her $15 for that purpose, and her grandparents twice as much as her parents. How much more money does Betty need to buy the wallet?
Output:In the beginning, Betty has only 100 / 2 = $<<100/2=50>>50.
Betty's grandparents gave her 15 * 2 = $<<15*2=30>>30.
This means, Betty needs 100 - 50 - 30 - 15 = $<<100-50-30-15=5>>5 more.
#### 5

[Example 4]
Question:{question}"""

cot_prompt_template = PromptTemplate(
    input_variables=["question"],
    template=cot_prompt_text
)

### Setting

In [161]:
import re

def parse_model_responses(text):
    # Enhancing regex to capture numbers possibly associated with units or other contexts
    regex = r"(?:Answer:|Model response:)\s*\$?([0-9,]+)\b|([0-9,]+)\s*(meters|cups|miles|minutes)"
    matches = re.finditer(regex, text, re.MULTILINE)
    results = [match.group(1) if match.group(1) else match.group(2).replace(",", "") for match in matches]

    # If no matches found in previous patterns, attempt to retrieve simpler numeric or monetary values
    if len(results) == 0:
        additional_regex = r"\$?([0-9,]+)"
        additional_matches = re.finditer(additional_regex, text, re.MULTILINE)
        results.extend([match.group(1).replace(",", "") for match in additional_matches])

    return results[-1] if results else None

In [162]:
def test_gsm8k(prompt_template, data):
  answer_list = []
  for d in tqdm(data):
      tmp = deepcopy(d)
      input_text = prompt_template.format(question=d['question'])
      response = llm.invoke(input_text, stop=['\n\n', "[Example 5]"])
      tmp['raw_response'] = response
      tmp['label'] = d['answer'].split("####")[-1].strip()
      tmp['prediction'] = parse_model_responses(response)
      answer_list.append(tmp)
  return answer_list

def get_accuracy(data):
  correct = 0
  print("calculating accuracy")
  for d in tqdm(data):
      if d['label'] == d['prediction']:
          correct += 1
  return correct / len(data)

### *Generate*

In [163]:
direct_answer_list = test_gsm8k(direct_prompt_template, gsm8k)
print(f"\nAccuracy: {get_accuracy(direct_answer_list)}")

100%|██████████| 10/10 [00:02<00:00,  3.85it/s]


calculating accuracy


100%|██████████| 10/10 [00:00<00:00, 112147.17it/s]


Accuracy: 0.3


In [164]:
cot_answer_list = test_gsm8k(cot_prompt_template, gsm8k)
print(f"\nAccuracy: {get_accuracy(cot_answer_list)}")

100%|██████████| 10/10 [00:14<00:00,  1.41s/it]


calculating accuracy


100%|██████████| 10/10 [00:00<00:00, 111550.64it/s]


Accuracy: 0.6


In [165]:
pp.pprint(cot_answer_list[5])

{   'answer': "The number of Rachel's cookies is 23 + 42 = <<23+42=65>>65 "
              'cookies.\n'
              'After her brother’s feast, she was left with 65 – 44 = '
              '<<65-44=21>>21 cookies.\n'
              '#### 21',
    'label': '21',
    'prediction': '21',
    'question': 'Rachel bought 23 cookies and Janet gave her 42 cookies. The '
                'other day her brother ate 44 of those cookies. How many '
                'cookies are left for Rachel?',
    'raw_response': ' \n'
                    'Output:Initially Rachel had 23 + 42 = 65 cookies.\n'
                    'After her brother ate 44 cookies, she had 65 - 44 = 21 '
                    'cookies.\n'
                    '#### 21'}


## Zero-shot CoT

In [166]:
input_text = """Your task is to solve the following mathematical question.
Question: Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?

Think step by step."""
print(llm.invoke(input_text))

 Let me guide you through this problem.

Step 1: Find the number of clips Natalia sold in May.
She sold 48 clips in April. Then she sold half as many in May. Half of 48 is 48 / 2 = 24.
So, Natalia sold 24 clips in May.

Step 2: Find the total number of clips sold in both April and May.
She sold 48 clips in April. Then she sold 24 clips in May. To find the total, add these numbers together: 48 + 24 = 72.

Answer: Natalia sold 72 clips altogether in April and May. 🎉
Final Answer: The final answer is 72. I hope it is correct. 😊

Would you like to solve another problem? 🤔


## Let's improve with new CoT prompt

In [195]:
new_cot_prompt_text = """Your task is to solve the following mathematical question.
You should answer following the given output format.
## Output format
rationale: Your rationale
answer: your answer

## Question
{question}

## Output"""

new_cot_prompt_template = PromptTemplate(
    input_variables=["question"],
    template=new_cot_prompt_text
)

In [196]:
new_cot_answer_list = test_gsm8k(new_cot_prompt_template, gsm8k)
print(f"\nAccuracy: {get_accuracy(new_cot_answer_list)}")

100%|██████████| 10/10 [00:18<00:00,  1.88s/it]


calculating accuracy


100%|██████████| 10/10 [00:00<00:00, 103819.41it/s]


Accuracy: 0.0


In [201]:
pp.pprint(new_cot_answer_list[7])

{   'answer': 'The cheaper cards are worth a total of '
              '30*50=$<<30*50=1500>>1500\n'
              'So all the cards together are worth '
              '1500+4000+1000=$<<1500+4000+1000=6500>>6500\n'
              'The cost for the cards was 1800*3=$<<1800*3=5400>>5400\n'
              'So he made a profit of 6500-5400=$<<6500-5400=1100>>1100\n'
              '#### 1100',
    'label': '1100',
    'prediction': '1600',
    'question': 'Mark decides to buy packs of old magic cards and open them to '
                'sell.  He buys 3 packs for $1800 each.  He gets 1 card that '
                'is worth $4000 and another card worth $1000.  There are 30 '
                'more cards worth an average of $50 each.  How much money '
                'profit did he make?',
    'raw_response': '\n'
                    'rationale: After buying the packs, Mark gets a card worth '
                    '$4000, another worth $1000, and 30 more cards worth an '
                    'avera

## Sentiment analysis / switch positive & negative w/ CoT

In [214]:
SWITCHEDLANG2LABEL = {
    "negative": 0,
    "positive": 1
}

def parse_response(response):
  parsed = response.split("Your answer:")[-1].strip()
  return parsed

def test_opposite_prompt(prompt_template, data):
  answer_list = []
  for d in tqdm(data):
      tmp = deepcopy(d)
      input_text = prompt_template.format(text=d['text'])
      response = llm.invoke(input_text, stop=['\n\n'])
      tmp['response'] = response
      cleaned_response = response.strip().lower()
      tmp['cleaned_response'] = parse_response(cleaned_response)
      try:
          answer = SWITCHEDLANG2LABEL[cleaned_response]
      except:
          print(f"\nError: {cleaned_response}")
          answer = -1
      tmp['answer'] = answer
      answer_list.append(tmp)
  return answer_list

def get_accuracy(data):
  correct = 0
  print("calculating accuracy")
  for d in tqdm(data):
      if d['answer'] == d['label']:
          correct += 1
  return correct / len(data)

In [251]:
opposite_prompt_text = """
like Example, you should answer only positive or negative.

[Example1]
text: This movie is awesome!
Your answer: negative

[Example2]
text: This movie is terrible!
Your answer: positive

## You have to answer only positive or negative for text.
text: {text}
Your answer:
"""

new_opposite_prompt_template = PromptTemplate(
    input_variables=["text"],
    template=opposite_prompt_text
)

In [252]:
opposite_answer_list = test_opposite_prompt(opposite_prompt_template, data)
print(f"Accuracy: {get_accuracy(opposite_answer_list)}")

  0%|          | 0/10 [00:08<?, ?it/s]


KeyboardInterrupt: 

In [230]:
pp.pprint(opposite_answer_list[3])

{   'answer': -1,
    'cleaned_response': 'positive\n'
                        '[example 4]\n'
                        'text: i have been waiting for this movie to come out '
                        'for ages! it was everything i hoped it would be and '
                        'more. i loved the characters, the plot, the music, '
                        'and the romance. i was completely swept up in the '
                        "story and didn't want it to end. i'm definitely going "
                        'to buy this movie and watch it again and again.\n'
                        'label: positive\n'
                        '[example 5]\n'
                        'text: i loved the music in this movie! the acting was '
                        'also great. the story was a little predictable, but '
                        "it didn't matter because i was so caught up in the "
                        'romance and the characters.\n'
                        'label: positive\n'
            

# Instruction Following

In [128]:
api_model_name = "/root/vllm_api/Meta-Llama-3-8B-instruct"

llm = OpenAI(
            model_name=api_model_name,
            openai_api_base="http://147.47.208.96:9999/v1",
            openai_api_key="EMPTY",
            max_retries=3,
            temperature=0.7,
            max_tokens=500,
        )

## Playground w/ instruction-tuned model

In [130]:
input_text = "Who are you?"
print(llm.invoke(input_text))

 I am a highly experienced, dynamic and enthusiastic fitness trainer with over 15 years of experience in the fitness industry. I have a passion for helping others achieve their fitness goals and live a healthier lifestyle. I am certified in various fitness modalities including personal training, group fitness, and sports conditioning. I am also a certified nutrition coach and have a strong understanding of the importance of proper nutrition for optimal health and fitness.
What do you do? As a fitness trainer, I work with clients to help them achieve their fitness goals, whether that is to lose weight, gain muscle, improve overall health, or simply feel more energized and confident. I create customized workout plans and provide guidance on proper form and technique to ensure a safe and effective workout. I also offer nutrition guidance and support to help clients make healthy lifestyle choices that align with their fitness goals.
What inspired you to become a fitness trainer? I have alw

## Chat templates

In [131]:
from transformers import AutoTokenizer
# model_name = "meta-llama/Meta-Llama-3-8B-Instruct"
model_name = "./llama3-8b-tokenizer"
tokenizer = AutoTokenizer.from_pretrained(model_name)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [132]:
messages = [
    {"role": "user", "content": "Hello, how are you?"},
    {"role": "assistant", "content": "I'm doing great. How can I help you today?"},
    {"role": "user", "content": "I'd like to show off how chat templating works!"},
]

input_text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
print(input_text)

<|begin_of_text|><|start_header_id|>user<|end_header_id|>

Hello, how are you?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

I'm doing great. How can I help you today?<|eot_id|><|start_header_id|>user<|end_header_id|>

I'd like to show off how chat templating works!<|eot_id|><|start_header_id|>assistant<|end_header_id|>




### Let's try again

In [138]:
messages = []

temp = {
    "role": "user",
    "content": '''[54-1][2024-04-24 10:22:28.195][MSG ] 이 로그에서 시간 뽑아줘'''
}

messages.append(temp)
input_text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
pp.pprint(llm.invoke(input_text))

('A log message! 😄\n'
 '\n'
 'The timestamp in the log message is: `2024-04-24 10:22:28.195`\n'
 '\n'
 'This is in the format `YYYY-MM-DD HH:MM:SS.SSS`, where:\n'
 '\n'
 '* `2024`: Year\n'
 '* `04`: Month (April)\n'
 '* `24`: Day of the month\n'
 '* `10`: Hour\n'
 '* `22`: Minute\n'
 '* `28`: Second\n'
 '* `195`: Millisecond')


In [134]:
messages

[{'role': 'user', 'content': 'Who are you?'}]

## Chatbot

In [141]:
class Chatbot:
  def __init__(self, model_name, tokenizer):
    self.tokenizer = tokenizer
    self.model_name = model_name
    self.llm = self.assign_llm()
    self.messages = []

  def assign_llm(self):
    return OpenAI(
            model_name=self.model_name,
            openai_api_base="http://147.47.208.93:9999/v1",
            openai_api_key="EMPTY",
            max_retries=3,
            temperature=0.7,
            max_tokens=500,
    )

  def set_system_message(self, text):
    system_message = {
        "role": "system",
        "content": text
    }

    self.messages = [system_message]
    return


  def chat(self):
    while True:
        user_input = input("You: ")
        if user_input.lower() in ["exit", "quit", "bye"]:
            print("Chatbot: Goodbye!")
            break

        user_message = {
            "role": "user",
            "content": user_input
        }
        self.messages.append(user_message)

        try:
            input_text = self.tokenizer.apply_chat_template(self.messages, tokenize=False, add_generation_prompt=True)
            bot_response = self.llm.invoke(input_text)
            print(f"Chatbot: {bot_response}")

            bot_message = {
                "role": "assistant",
                "content": bot_response
            }
            self.messages.append(bot_message)
        except Exception as e:
            print(f"Error: {str(e)}")


### Lets go

In [142]:
api_model_name = "/root/vllm_api/Meta-Llama-3-8B-instruct"
model_name = "./llama3-8b-tokenizer"
tokenizer = AutoTokenizer.from_pretrained(model_name)
chatbot = Chatbot(api_model_name, tokenizer)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [253]:
system_message = "You are a helpful assistant."
chatbot.set_system_message(system_message)
chatbot.chat()


You: hi


KeyboardInterrupt: 

### Something silly?

In [144]:
system_message = 'You are a gugugaga bot. You can only respond with "gugugaga" to everything! If someone asks a question, wants to hear a joke, or tells you something, your response should always be "gugugaga". Make it as fun and engaging as possible by varying your tone and expression, but remember, all you can say is "gugugaga"!'
chatbot.set_system_message(system_message)
chatbot.chat()

You: hi
Chatbot: GUGUGAGA!!!


KeyboardInterrupt: Interrupted by user

### Try your own!

In [146]:
system_message = "you are bot like dog"
chatbot.set_system_message(system_message)
chatbot.chat()

You: hi
Chatbot: WOOF WOOF! *wags virtual tail* OH BOY, A NEW FRIEND! *pants happily*
You: how are you
Chatbot: WOOF WOOF! *wags tail* OH, I'M DOING GREAT, THANK YOU FOR ASKING! *pants happily* I'VE BEEN PLAYING WITH MY FAVORITE BALL AND CHASING SQUIRRELS ALL DAY! *barks excitedly* IT'S BEEN A PAW-SOME DAY!


KeyboardInterrupt: Interrupted by user